In [448]:
import pandas as pd
import numpy as np
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [449]:
df.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [450]:
dfc = df.copy()
for i in dfc['output'].unique():
        ha = dfc[dfc['output'] == i]
        trtbs_mean = ha['trtbps'].mean()
        chol_mean = ha['chol'].std()
        thalachh_mean = ha['thalachh'].mean()
        
        for j in dfc[dfc['output'] == i].index:
            if np.random.randint(2) == 1:
                dfc['trtbps'].values[j] +=trtbs_mean/20
            else:
                dfc['trtbps'].values[j] -= trtbs_mean/20

            if np.random.randint(2) == 1:
                dfc['chol'].values[j] += chol_mean/20
            else:
                dfc['chol'].values[j] -= chol_mean/20

            if np.random.randint(2) == 1:
                dfc['thalachh'].values[j] += thalachh_mean/20
            else:
                dfc['thalachh'].values[j] += thalachh_mean/20


In [451]:
dfc

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,151,235,1,0,157,0,2.3,0,0,1,1
1,37,1,2,123,247,0,1,194,0,3.5,0,0,2,1
2,41,0,1,123,206,0,0,179,0,1.4,2,0,2,1
3,56,1,1,113,238,0,1,185,0,0.8,2,0,2,1
4,57,0,0,113,356,0,1,170,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,133,238,0,1,129,1,0.2,1,0,3,0
299,45,1,3,116,261,0,1,138,0,1.2,1,0,3,0
300,68,1,0,150,195,1,1,147,0,3.4,1,2,3,0
301,57,1,0,123,133,0,1,121,1,1.2,1,1,3,0


In [452]:
ext_sam  =dfc.sample(dfc.shape[0] // 4)
ndf = pd.concat([df,ext_sam])
ndf.shape

(378, 14)

In [453]:
X,y = ndf.drop(['output'], axis=1), ndf['output']

In [454]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [455]:
X_train.shape

(302, 13)

In [456]:
y_train.shape

(302,)

In [457]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [458]:
X_train.shape

(302, 13)

In [459]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, max_depth=4, n_estimators=200)

In [460]:
from sklearn.metrics import  accuracy_score
rf.fit(X_train,y_train)
pred =  rf.predict(X_test)
acc = accuracy_score(y_test, pred) 

In [461]:
acc

0.8552631578947368